<a href="https://colab.research.google.com/github/NehaAgg245/ML-git/blob/master/Campus_load_hourly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving L1_Campus_load_2014-2015_v3.csv to L1_Campus_load_2014-2015_v3.csv


In [0]:
import pandas as pd 
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
from pandas import concat
from itertools import islice
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import io
from itertools import islice

# Import data
campus_data = pd.read_csv(io.StringIO(uploaded['L1_Campus_load_2014-2015_v3.csv'].decode('utf-8')))
campus_data['Timestamp'] = pd.to_datetime(campus_data['Timestamp'])
campus_data.set_index('Timestamp', inplace = True)

utd = campus_data.iloc[:, -1:]



In [0]:
def outliers_iqr(utd):
	utd_updated = pd.DataFrame(columns = ['Total'])
	for year in np.array_split(utd,2):
		quartile_1, quartile_3 = np.percentile(year,[25,75])
		iqr = quartile_3 - quartile_1
		lower_bound = quartile_1 - (iqr * 1.5)
		upper_bound = quartile_3 + (iqr * 1.5)
		for i in year.index:
			if year.at[i,'Total'] < lower_bound:
				low_val = year.at[i,'Total']
				year['Total'].replace(low_val,lower_bound,inplace = True)
			elif year.at[i,'Total'] > upper_bound:
				up_val = year.at[i,'Total']
				year['Total'].replace(up_val,upper_bound,inplace = True)
		utd_updated = utd_updated.append(year)
	return utd_updated

utd = outliers_iqr(utd)

#last time step (t-1) as the input and the observation at the current time step (t) as the output
def timeseries_to_supervised(data, lag = 1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis = 1)
	df.fillna(0,inplace = True)
	return df

#Calculating difference time series to get a stationarity in data
def difference(dataset, interval = 1):
	diff = list()
	for index, row in islice(dataset.iterrows(), interval, None):
		i = dataset.index.get_loc(index)
		i = i-interval
		value = dataset.at[index, 'Total'] - dataset.iloc[i, dataset.columns.get_loc('Total')]
		diff.append(value)
	return Series(diff)

#Invert differenced value to get back into the original scale
def inverse_difference(history, yhat, interval = 1):
	return yhat + history[-interval]

differenced = difference(utd, 1)

# inverted = list()
# for i in range(len(differenced)):
# 	value = inverse_difference(utd, differenced[i], len(utd) - i)
# 	inverted.append(value)
# inverted = Series(inverted)

#Transform scale 
def scale(train, test):
	scaler = MinMaxScaler(feature_range = (-1,1))
	scaler = scaler.fit(train)
	train = train.reshape(train.shape[0], test.shape[1])
	train_scaled = scaler.transform(train)
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

#inverting scaling for a transform value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = np.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler. inverse_transform(array)
	return inverted[0,-1]

#fit an lstm network tp training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful = True))
	model.add(Dense(1, activation = 'linear'))
	model.compile(loss = 'mean_squared_error', optimizer = 'adam')
	for i in range(nb_epoch):
		model.fit(X, y , epochs = 1, batch_size= batch_size, verbose = 0, shuffle = False)
		model.reset_states()
	return model

#make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X= X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size = batch_size)
	return yhat[0,0]


supervised = timeseries_to_supervised(differenced, 1)
supervised_values = supervised.values
train, test = supervised_values[0:-8760],supervised_values[-8760:]

scaler, train_scaled, test_scaled = scale(train, test)

In [0]:

lstm_model = fit_lstm(train_scaled, 1, 10, 4)


In [21]:
df_test = utd[-8760:]
arr_test = df_test.values
for index, row in islice(df_test.iterrows(),0, 24):
  r = df_test.index.get_loc(index)
  test_hr = arr_test[r:r+24]
  test1 = test_hr[0]
  test2 = test_hr[1]
  diff1 = test_scaled[0, 0]
  temp = test2
  test_matrix = np.zeros(shape = (len(test_hr)-2,2))
  test_matrix[0,0] = diff1
  predictions = list()
  for i in range(len(test_hr)-2):
    X,y = test_matrix[i,0:-1], test_matrix[i,-1]
    yhat = forecast_lstm(lstm_model,1,X)
    yhat_in = invert_scale(scaler, X, yhat)
    yhat_diff = yhat_in + temp
    predictions.append(temp)
    test_matrix[i,1] = yhat
    if i != (len(test_hr)-3):
      test_matrix[i+1,0] = yhat
    temp = yhat_diff
    #print(temp, test_hr[i+2])
  rmse = sqrt(mean_squared_error(arr_test[r+2:r+24], predictions))
  print('Test RMSE: %.3f' %rmse)

Test RMSE: 264.498
Test RMSE: 269.994
Test RMSE: 267.260
Test RMSE: 263.999
Test RMSE: 211.764
Test RMSE: 209.478
Test RMSE: 127.204
Test RMSE: 209.597
Test RMSE: 124.117
Test RMSE: 146.405
Test RMSE: 158.481
Test RMSE: 169.480
Test RMSE: 179.636
Test RMSE: 184.142
Test RMSE: 190.892
Test RMSE: 187.186
Test RMSE: 182.504
Test RMSE: 243.390
Test RMSE: 264.527
Test RMSE: 240.547
Test RMSE: 224.541
Test RMSE: 202.798
Test RMSE: 156.284
Test RMSE: 234.979
